# Single Shot Detector Training Models


**Install TensorFlow and other dependencies**

In [1]:
!pip install --upgrade pip
!pip install --upgrade protobuf 

     |████████████████████████████████| 1.6MB 3.9MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 1.0 MB 4.0 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [1]:
%tensorflow_version 1.15
import tensorflow as tf
print(tf.__version__)

!pip install numpy

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.2


**Check GPU status**

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

**Mount Google Drive folder**

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

# change to working tensorflow directory on the drive
#%cd '/content/gdrive/My Drive/models/'

Mounted at /content/gdrive


In [5]:
# I change My\ Drive/ to mydrive for easier

!ln -s /content/gdrive/My\ Drive/ /mydrive

In [6]:
# I am controlling the contents of I created yolov folder

!ls /content/gdrive/MyDrive/models

AUTHORS     community	     ISSUES.md	official  README.md
CODEOWNERS  CONTRIBUTING.md  LICENSE	orbit	  research


**Install protobuf and compile, install setup.py**

In [7]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
%cd /content/gdrive/MyDrive/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/gdrive/MyDrive/models/research/:/content/gdrive/MyDrive/models/research/slim'

!python /content/gdrive/MyDrive/models/research/slim/setup.py build
!python /content/gdrive/MyDrive/models/research/slim/setup.py install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 34 not upgraded.
Need to get 1,614 kB of archives.
After this operation, 8,907 kB of additional disk space will be used.
Get:1 htt

**Check remaining GPU time**

In [ ]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Time remaining for this session is:  11.735525415605968


# ***Start training***

In [ ]:
!pip install tf_slim
%cd /content/gdrive/MyDrive/models/research/object_detection
os.environ['PYTHONPATH'] += ':/content/gdrive/MyDrive/models/research/:/content/gdrive/MyDrive/models/research/slim'

!python train.py --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v1_pets.config --logtostderr

/content/gdrive/MyDrive/models/research/object_detection
Instructions for updating:
Use object_detection/model_main.py.
W0504 12:32:23.164469 140570619799424 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/absl/app.py:251: main (from __main__) is deprecated and will be removed in a future version.
Instructions for updating:
Use object_detection/model_main.py.
Instructions for updating:
Please switch to tf.train.create_global_step
W0504 12:32:23.180333 140570619799424 deprecation.py:323] From /content/gdrive/MyDrive/models/research/object_detection/legacy/trainer.py:265: create_global_step (from tf_slim.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Please switch to tf.train.create_global_step
INFO:tensorflow:Reading unweighted datasets: ['data/train.record']
I0504 12:32:23.202529 140570619799424 dataset_builder.py:163] Reading unweighted datasets: ['data/train.record']
INFO:tensorflow:Reading record datasets for input fi

***Export inference graph***

In [ ]:
#  .ckpt needs to be updated every time to match last .ckpt generated
#  .config needs to be updated when changing model
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/ssd_mobilenet_v1_pets.config --trained_checkpoint_prefix training/model.ckpt-6325 --output_directory new_graph

Instructions for updating:
Please use `layer.__call__` method instead.
W0504 13:42:09.772566 140570714503040 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0504 13:42:11.701059 140570714503040 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0504 13:42:11.748720 140570714503040 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0504 13:42:11.793092 140570714503040 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

Zip file in Google Drive

In [ ]:
!zip -r model_graph.zip new_graph

  adding: new_graph/ (stored 0%)
  adding: new_graph/model.ckpt.data-00000-of-00001 (deflated 7%)
  adding: new_graph/model.ckpt.index (deflated 67%)
  adding: new_graph/checkpoint (deflated 42%)
  adding: new_graph/model.ckpt.meta (deflated 93%)
  adding: new_graph/frozen_inference_graph.pb (deflated 9%)
  adding: new_graph/saved_model/ (stored 0%)
  adding: new_graph/saved_model/variables/ (stored 0%)
  adding: new_graph/saved_model/saved_model.pb (deflated 9%)
  adding: new_graph/pipeline.config (deflated 69%)


## **Test**


*   **40 images tested and detected.**
*   **Some videos tested**



***Run the code "custom_model_images" to test the images.***

In [ ]:
!python /content/gdrive/MyDrive/models/research/object_detection/custom_model_images.py

Instructions for updating:
non-resource variables are not supported in the long term
2021-05-04 13:42:53.020615: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-05-04 13:42:53.025838: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-05-04 13:42:53.026766: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2021-05-04 13:42:53.027072: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-05-04 13:42:53.028946: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-05-04 13:42:53.030480: I tensorflow/stream_executor/pla

***If it is not working, run the below code***

In [ ]:
# !pip install lvis

***Run the code "custom_model_video" to test the video.***

In [59]:
!python /content/gdrive/MyDrive/models/research/object_detection/custom_model_video.py

(800, 600) (600, 800, 3)
(800, 600) (600, 800, 3)
(800, 600) (600, 800, 3)
(800, 600) (600, 800, 3)



2021-05-11 10:31:55.823021: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-05-11 10:31:55.835081: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-05-11 10:31:55.835157: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (41c663cf8573): /proc/driver/nvidia/version does not exist
2021-05-11 10:31:55.842791: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-05-11 10:31:55.843080: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55dec813e840 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-05-11 10:31:55.843118: I tensorflow/compiler/xla/service/service.cc:176]   Str

In [60]:
!python /content/gdrive/MyDrive/models/research/object_detection/custom_model_video.py

(800, 600) (600, 800, 3)
(800, 600) (600, 800, 3)
(800, 600) (600, 800, 3)
(800, 600) (600, 800, 3)



2021-05-11 10:35:35.591702: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-05-11 10:35:35.603048: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-05-11 10:35:35.603119: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (41c663cf8573): /proc/driver/nvidia/version does not exist
2021-05-11 10:35:35.609831: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-05-11 10:35:35.610104: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55b538986840 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-05-11 10:35:35.610141: I tensorflow/compiler/xla/service/service.cc:176]   Str

In [61]:
!python /content/gdrive/MyDrive/models/research/object_detection/custom_model_video.py

(800, 600) (600, 800, 3)
(800, 600) (600, 800, 3)
(800, 600) (600, 800, 3)
(800, 600) (600, 800, 3)



2021-05-11 10:38:55.713551: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-05-11 10:38:55.724463: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-05-11 10:38:55.724531: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (41c663cf8573): /proc/driver/nvidia/version does not exist
2021-05-11 10:38:55.729940: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-05-11 10:38:55.730169: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x559a6b1a2840 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-05-11 10:38:55.730202: I tensorflow/compiler/xla/service/service.cc:176]   Str


***Viewed on tensorboard***

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir '/content/gdrive/MyDrive/models/research/object_detection/training'

***Summary metrics of model***

In [ ]:
!python /content/gdrive/MyDrive/models/research/object_detection/model_main.py --pipeline_config_path=/content/gdrive/MyDrive/models/research/object_detection/training/ssd_mobilenet_v1_pets.config --model_dir=/content/gdrive/MyDrive/models/research/object_detection/new_graph --eval_training_data=True --checkpoint_dir=/content/gdrive/MyDrive/models/research/object_detection/training --run_once=True


W0505 12:08:24.060092 140315987621760 model_lib.py:812] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: None
I0505 12:08:24.060441 140315987621760 config_util.py:552] Maybe overwriting train_steps: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0505 12:08:24.060575 140315987621760 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0505 12:08:24.060707 140315987621760 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0505 12:08:24.060897 140315987621760 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0505 12:08:24.061076 140315987621760 model_lib.py:828] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to_t

***Confusion matrix in csv format***

In [ ]:
!python /content/gdrive/MyDrive/models/research/object_detection/confusion_matrix.py --detections_record=/content/gdrive/MyDrive/models/research/object_detection/data/out.record --label_map=/content/gdrive/MyDrive/models/research/object_detection/training/object-detection.pbtxt --output_path=/content/gdrive/MyDrive/models/research/object_detection/data/confusion_matrix.csv



Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
W0506 12:55:39.609995 139917550897024 deprecation.py:323] From /content/gdrive/MyDrive/models/research/object_detection/confusion_matrix.py:38: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Processed 100 images
Processed 200 images
Processed 300 images
Processed 400 images
Processed 429 images

Confusion Matrix:
[[235.   0. 196.]
 [ 20.  20. 221.]
 [ 13.   3.   0.]] 

    category  ...  recall_@0.5IOU
0  flare oil  ...        0.545244
1      smoke  ...        0.076628

[2 rows x 3 columns]


***To find "out.record" run below code.***

In [54]:
!python /content/gdrive/MyDrive/models/research/object_detection/inference/infer_detections.py --input_tfrecord_paths=/content/gdrive/MyDrive/models/research/object_detection/data/test.record --output_tfrecord_path=/content/gdrive/MyDrive/models/research/object_detection/data/out.record --inference_graph=/content/gdrive/MyDrive/models/research/object_detection/new_graph/frozen_inference_graph.pb


2021-05-11 10:23:44.717048: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-05-11 10:23:44.732588: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-05-11 10:23:44.732659: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (41c663cf8573): /proc/driver/nvidia/version does not exist
2021-05-11 10:23:44.749283: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2021-05-11 10:23:44.749682: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56286240cf40 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-05-11 10:23:44.749730: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
INFO:tensorflow:Reading input from 1 files
I0511 10:23:44